In [ ]:
# Google Cloud BigQuery
from google.cloud import bigquery

# Reddit API
import praw
import requests

# Data Manipulation & Exploration
import pandas as pd
import datetime
import time

# Access Credentials
import json
import os

In [ ]:
# Initialize Client Object
client = bigquery.Client()

In [ ]:
# Path to Reddit API Credentials
credentials = 'client_secrets.json'

# Read Credentials from JSON file
with open(credentials) as f:
    creds = json.load(f)

In [ ]:
# Python Reddit API Wrapper
reddit = praw.Reddit(client_id=creds['client_id'],
                     client_secret=creds['client_secret'],
                     user_agent=creds['user_agent'],
                     redirect_uri=creds['redirect_uri'],
                     refresh_token=creds['refresh_token'])

In [ ]:
# Create an Empty DataFrame for Result Storage
bigdata = pd.DataFrame()

# Provide List of Different Genres
genres = ['new','hot','rising','top']

# Define Search Parameter
search = reddit.subreddit('Kenya')

# Loop through Genres while Extracting Posts
for genre in genres:
    posts = []
    for post in getattr(search, genre)(limit=1000):
        created_at_datetime = datetime.datetime.fromtimestamp(post.created)
        today_date = datetime.datetime.today()
        genr = genre
        posts.append([genr, post.title, post.score, post.id, post.subreddit, post.url, 
                    post.num_comments, post.selftext, created_at_datetime, today_date, search.subscribers
                    ])
    data = pd.DataFrame(
        posts,
        columns=[
            'genre', 'title', 'score', 'id', 'subreddit', 'url', 'num_comments', 
            'body', 'created', 'today_date','subscribers'
                ])
    
    # Concatenate 
    bigdata = pd.concat([bigdata,data],ignore_index=True)

bigdata.to_csv('reddit.csv',index=False)

In [ ]:
# Import Data from CSV
data = pd.read_csv('reddit.csv')

# Define Table Id
table_id = 'project-adrian-julius-aluoch.cronjobs.reddit_kenya'

# Load Extracted Data into BigQuery
job = client.load_table_from_dataframe(data,table_id)
while job.state != 'DONE':
    time.sleep(4)
    job.reload()
    print(job.state)

# Delete CSV file
os.remove('reddit.csv')

In [ ]:
# Define SQL Query to Retrieve All Records from BigQuery
sql = (
    'SELECT *'
    'FROM `cronjobs.reddit_kenya`'
      )

# Run SQL Query
data = client.query(sql).to_dataframe()

# Drop Duplicated Records
data.drop_duplicates(subset=['title'],inplace=True)

# Replace Original BigQuery Table 
client.delete_table(table_id)

# Upload New BigQuery Table
job = client.load_table_from_dataframe(data,table_id)
while job.state != 'DONE':
    time.sleep(1)
    job.reload()
    print(job.state)